# AI Agents with Generative AI Hub

Everyone is talking about AI Agents and how they are going to make LLMs more powerful. AI Agents are basically systems that have a set of tools that they can use and the LLM is the agents brain to decide which tool to use when. Some systems are more agentic than others, depending on the amount of autonomy and decision-making power they possess. 

If you want to know more about AI Agents and how to build one from scratch, check out this [Devtoberfest session](https://community.sap.com/t5/devtoberfest/getting-started-with-agents-using-sap-generative-ai-hub/ev-p/13865119).

👉 Before you start you will need to add the constant `LLM_DEPLOYMENT_ID` to [variables.py](variables.py). You can use the `gpt-4o-mini` model again that we deployed earlier. You find the deployment ID in SAP AI Launchpad.

In [1]:
import json
import os
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)
 
# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "grounding"
    }
 
    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)
 
# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )
 
ai_core_client = create_ai_core_client(credCF)

https://israel-fsvdxbsq.authentication.eu11.hana.ondemand.com/oauth/token
sb-49ec08a9-d325-4480-9418-ad8801558203!b28574|aicore!b18
96d2ba69-3289-4190-ad82-c174e50f9f17$8C_adlgCYD6AscPgIKtLXJkIj1AL6i8p9Opw1JJZ0o8=
https://api.ai.prodeuonly.eu-central-1.aws.ml.hana.ondemand.com/v2
grounding


### Import the packages you want to use

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings


We will give the agent different tools to use. The first tool will be [access to Wikipedia](https://python.langchain.com/docs/integrations/tools/wikipedia/). This way instead of answering from it's training data, the model will check on Wikipedia articles first.

In [3]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Let's check if it works
print(wikipedia.run("Bangalore"))

# Assign wikipedia to the set of tools
tools = [wikipedia]

Page: Bengaluru
Summary: Bengaluru, also known as Bangalore (its official name until 1 November 2014), is the capital and largest city of the southern Indian state of Karnataka. As per the 2011 census, the city had a population of 8.4 million, making it the third most populous city in India and the most populous in South India. The Bengaluru metropolitan area had a population of around 8.5 million, making it the fifth most populous urban agglomeration in the country. It is located near the center of the Deccan Plateau, at a height of 900 m (3,000 ft) above sea level. The city is known as India's "Garden City", due to its parks and greenery.
Archaeological artifacts indicate that the human settlement in the region happened as early as 4000 BCE. The first mention of the name "Bengalooru" is from an old Kannada stone inscription from 890 CE found at the Nageshwara Temple. From 350 CE, it was ruled by the Western Ganga dynasty, and in the early eleventh century, the city became part of the

In [4]:
# For agents the initial prompt including all the instructions is very important. 
# LangChain already has a good set of prompts that we can reuse here.
prompt = hub.pull("hwchase17/react")
prompt

c:\Users\C5385222\AppData\Local\anaconda3\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [5]:
# Create a chat model instance which will act as the brain of the agent
llm = ChatOpenAI(deployment_id="d6dd6e483c92f88c")

# Create an agent with the llm, tools and prompt
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Let's ask try the agent and ask about Bangalore
agent_executor.invoke({"input": "Tell me about Bangalore!"})



> Entering new AgentExecutor chain...
Bangalore is a significant city in India, known for its technology sector. I'll gather more information about its history, culture, and significance. 
Action: wikipedia
Action Input: BangalorePage: Bengaluru
Summary: Bengaluru, also known as Bangalore (its official name until 1 November 2014), is the capital and largest city of the southern Indian state of Karnataka. As per the 2011 census, the city had a population of 8.4 million, making it the third most populous city in India and the most populous in South India. The Bengaluru metropolitan area had a population of around 8.5 million, making it the fifth most populous urban agglomeration in the country. It is located near the center of the Deccan Plateau, at a height of 900 m (3,000 ft) above sea level. The city is known as India's "Garden City", due to its parks and greenery.
Archaeological artifacts indicate that the human settlement in the region happened as early as 4000 BCE. The first ment

{'input': 'Tell me about Bangalore!',
 'output': 'Bengaluru, formerly known as Bangalore, is the capital of Karnataka, India, and one of the country\'s most populous cities. It has a rich history dating back to 4000 BCE with notable periods of rule by various dynasties, including the Western Ganga, Chola, Hoysala, and Vijayanagara empires. In 1537, Kempe Gowda I established a fort that laid the foundation for the modern city. It later came under British control and became part of the Princely State of Mysore. Bengaluru is often referred to as the "Silicon Valley of India" due to its significant information technology industry and is known for its parks, earning it the nickname "Garden City." As of 2023, it has a burgeoning economy with an estimated GDP of $359.9 billion, making it one of the fastest-growing cities in India.'}

## Give the agent access to our HANA vector store
Now the agent can use Wikipedia but wouldn't it be nice if the agent could pick between resources and decide when to use what?

In [6]:
def get_hana_connection():
    conn = dbapi.connect(
        address='ec41b786-96de-467b-9ff5-db725945f89c.hna0.prod-us10.hanacloud.ondemand.com',
        port='443',
        user='DBADMIN',
        password='9hEW4UK86Fdt',
        encrypt=True
    )
    return conn

In [7]:
from langchain_community.vectorstores.hanavector import HanaDB
from langchain.tools.retriever import create_retriever_tool
from hdbcli import dbapi
# # connect to HANA instance
# connection = init_env.connect_to_hana_db()
# connection.isconnected()

# Reference which embedding model, DB connection and table to use
embeddings = OpenAIEmbeddings(deployment_id="d1ae8a45e3d60115")
db = HanaDB(
    embedding=embeddings, connection=get_hana_connection(), table_name="EMBEDDINGS_CODEJAM_"+"Testing"
)

# Create a retriever instance of the vector store
retriever = db.as_retriever(search_kwargs={"k": 2})

# We need to add a description to the retriever so that the llm knows when to use this tool.
retriever_tool = create_retriever_tool(
    retriever,
    "SAP orchestration service docu",
    "Search for information SAP's orchestration service. For any questions about generative AI hub and sap's orchestration service, you must use this tool!",
)

## Ask the Agent

You can ask the agent anything in general and it will try to find an entry from Wikipedia, unless you ask about SAP's orchestration service. Then it will get the information from SAP HANA vector store, where it holds the documentation, instead of Wikipedia.

In [8]:
# Now let's add the retriever as a tool
tools = [wikipedia, retriever_tool]

# And create the agent again with the two tools, wikipedia and the HANA vector store (retriever)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# First ask: What is Data Masking?
# Then ask: What is Data Masking in SAP's Generative AI Hub?
# Pay attention to the response! Do you see what is happening now?
agent_executor.invoke({"input": "What is data masking?"})



> Entering new AgentExecutor chain...
Data masking refers to the process of obfuscating specific data within a database to protect it from unauthorized access while maintaining its usability for tasks like software testing or user training. This is typically accomplished by replacing sensitive information with altered values that are not identifiable but retain the same format and characteristics as the original data.

Action: wikipedia
Action Input: "Data masking"

c:\Users\C5385222\AppData\Local\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\C5385222\AppData\Local\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Data masking
Summary: Data masking or data obfuscation is the process of modifying sensitive data in such a way that it is of no or little value to unauthorized intruders while still being usable by software or authorized personnel. Data masking can also be referred as anonymization, or tokenization, depending on different context.
The main reason to mask data is to protect information that is classified as personally identifiable information, or mission critical data. However, the data must remain usable for the purposes of undertaking valid test cycles. It must also look real and appear consistent. It is more common to have masking applied to data that is represented outside of a corporate production system. In other words, where data is needed for the purpose of application development, building program extensions and conducting various test cycles. It is common practice in enterprise computing to take data from the production systems to fill the data component, required for t

{'input': 'What is data masking?',
 'output': "Data masking or data obfuscation is the process of modifying sensitive data so that it holds little or no value for unauthorized intruders while remaining usable by authorized personnel or software. This involves replacing sensitive information with altered values that maintain the original data's format and characteristics. Data masking is primarily used to protect personally identifiable information and mission-critical data, especially in non-production environments for tasks like software testing and user training. It helps mitigate the risk of data breaches by ensuring that individuals without proper security clearance do not have access to sensitive information."}